这是 CE 函数的测试文件

In [ ]:
import scipy.io
import numpy as np
from scipy import ndimage

In [118]:
# 读取 mat 文件，打印一下变量名称

mat_data = scipy.io.loadmat('data/CE_key_variables.mat')

for key in mat_data.keys():
    if not key.startswith('__'):  # 过滤掉MATLAB的元数据
        print(f"- {key}: {type(mat_data[key])}, shape: {mat_data[key].shape if hasattr(mat_data[key], 'shape') else 'N/A'}")

- I: <class 'numpy.ndarray'>, shape: (768, 576, 3)
- CE_gray: <class 'numpy.ndarray'>, shape: (768, 576)
- CE_by: <class 'numpy.ndarray'>, shape: (768, 576)
- CE_rg: <class 'numpy.ndarray'>, shape: (768, 576)
- sigma: <class 'numpy.ndarray'>, shape: (1, 1)
- semisaturation: <class 'numpy.ndarray'>, shape: (1, 1)
- t1: <class 'numpy.ndarray'>, shape: (1, 1)
- t2: <class 'numpy.ndarray'>, shape: (1, 1)
- t3: <class 'numpy.ndarray'>, shape: (1, 1)
- border_s: <class 'numpy.ndarray'>, shape: (1, 1)
- break_off_sigma: <class 'numpy.ndarray'>, shape: (1, 1)
- filtersize: <class 'numpy.ndarray'>, shape: (1, 1)
- x: <class 'numpy.ndarray'>, shape: (1, 20)
- Gauss: <class 'numpy.ndarray'>, shape: (1, 20)
- Gx: <class 'numpy.ndarray'>, shape: (1, 20)
- R: <class 'numpy.ndarray'>, shape: (768, 576)
- G: <class 'numpy.ndarray'>, shape: (768, 576)
- B: <class 'numpy.ndarray'>, shape: (768, 576)
- gray: <class 'numpy.ndarray'>, shape: (768, 576)
- by: <class 'numpy.ndarray'>, shape: (768, 576)
- rg:

In [119]:
I = mat_data['I']

print(I.shape)

(768, 576, 3)


In [120]:
def test(input_variable, variable_name, mat_file='data/CE_key_variables.mat'):
    """
    改进版测试函数：
    - 矩阵比较时显示具体数值
    - 标量比较时忽略(1,1)形状差异，简化输出
    """
    try:
        # 读取.mat文件
        mat_data = scipy.io.loadmat(mat_file)
        
        # 检查变量是否存在
        if variable_name not in mat_data:
            print(f"错误：变量 '{variable_name}' 不存在")
            return False
        
        mat_variable = mat_data[variable_name]
        
        # 转换为numpy数组
        input_array = np.array(input_variable)
        mat_array = np.array(mat_variable)
        
        # 判断是否为标量比较（输入是标量且mat变量是1x1数组）
        is_scalar_comparison = (input_array.shape == () and mat_array.shape == (1, 1)) or \
                              (input_array.shape == (1, 1) and mat_array.shape == ())
        
        if is_scalar_comparison:
            # 标量比较：简化处理
            print(f"比较标量值: 输入={input_variable}, Mat={(mat_array.item() if mat_array.size == 1 else mat_array)}")
            
            # 统一为标量进行比较
            input_scalar = input_array.item() if input_array.size == 1 else input_array
            mat_scalar = mat_array.item() if mat_array.size == 1 else mat_array
            
            are_equal = input_scalar == mat_scalar
        else:
            # 矩阵比较：显示详细信息
            print(f"输入变量形状: {input_array.shape}")
            print(f"Mat变量形状: {mat_array.shape}")
            # print(f"输入变量:\n{input_array}")
            # print(f"Mat变量:\n{mat_array}")
            
            # 检查形状是否相同
            if input_array.shape != mat_array.shape:
                print(f"形状不同：{input_array.shape} vs {mat_array.shape}")
                return False
            
            # 比较数值
            if np.issubdtype(input_array.dtype, np.floating) or np.issubdtype(mat_array.dtype, np.floating):
                are_equal = np.allclose(input_array, mat_array, rtol=1e-10, atol=1e-12)
            else:
                are_equal = np.array_equal(input_array, mat_array)
        
        # 输出结果
        if are_equal:
            print(f"✓ 变量 '{variable_name}' 数值相同")
            return True
        else:
            print(f"✗ 变量 '{variable_name}' 数值不同")
            if not is_scalar_comparison:
                # 只有矩阵比较时才显示详细差异
                if input_array.size <= 20:  # 小矩阵显示差异
                    diff = np.abs(input_array - mat_array)
                    print(f"差异矩阵:\n{diff}")
                else:
                    max_diff = np.max(np.abs(input_array - mat_array))
                    print(f"最大差异: {max_diff}")
            return False
            
    except Exception as e:
        print(f"出错: {e}")
        return False

In [121]:
# Basic parameters
sigma               = 3.25;
semisaturation      = 0.1; 
t1                  = 9.225496406318721e-004 *255; # 0.2353
t2                  = 8.969246659629488e-004 *255; # 0.2287
t3                  = 2.069284034165411e-004 *255; # 0.0528
border_s            = 20;

test(sigma , 'sigma')
test(semisaturation , 'semisaturation')
test(t1 , 't1')
test(t2 , 't2')
test(t3 , 't3')
test(border_s , 'border_s')

比较标量值: 输入=3.25, Mat=3.25
✓ 变量 'sigma' 数值相同
比较标量值: 输入=0.1, Mat=0.1
✓ 变量 'semisaturation' 数值相同
比较标量值: 输入=0.2352501583611274, Mat=0.2352501583611274
✓ 变量 't1' 数值相同
比较标量值: 输入=0.22871578982055193, Mat=0.22871578982055193
✓ 变量 't2' 数值相同
比较标量值: 输入=0.052766742871217985, Mat=0.052766742871217985
✓ 变量 't3' 数值相同
比较标量值: 输入=20, Mat=20
✓ 变量 'border_s' 数值相同


True

In [122]:
# Gaussian & LoG & Retification, Normalization(?)

break_off_sigma = 3
filtersize = break_off_sigma * sigma


x = np.arange(-filtersize, filtersize,1)  # MATLAB的-filtersize:1:filtersize
x = x.reshape(1, -1)
Gauss = (1 / (np.sqrt(2 * np.pi) * sigma)) * np.exp((x**2) / (-2 * sigma * sigma))
Gauss = Gauss / np.sum(Gauss)
Gx = (x**2 / sigma**4 - 1 / sigma**2) * Gauss  # LoG
Gx = Gx - np.sum(Gx) / x.size  # size(x,2) 在MATLAB中等于 x.size 在Python中
Gx = Gx / np.sum(0.5 * x * x * Gx)


test(break_off_sigma , 'break_off_sigma')
test(filtersize , 'filtersize')
test(x , 'x')
test(Gauss , 'Gauss')
test(Gx , 'Gx')

比较标量值: 输入=3, Mat=3
✓ 变量 'break_off_sigma' 数值相同
比较标量值: 输入=9.75, Mat=9.75
✓ 变量 'filtersize' 数值相同
输入变量形状: (1, 20)
Mat变量形状: (1, 20)
✓ 变量 'x' 数值相同
输入变量形状: (1, 20)
Mat变量形状: (1, 20)
✓ 变量 'Gauss' 数值相同
输入变量形状: (1, 20)
Mat变量形状: (1, 20)
✓ 变量 'Gx' 数值相同


True

In [123]:
# Color conversion

I = I.astype(np.float64)  # 等价于 MATLAB 的 double(I)
R = I[:, :, 0]            # 第一个通道 (Red)
G = I[:, :, 1]            # 第二个通道 (Green)  
B = I[:, :, 2]            # 第三个通道 (Blue)
gray = 0.299 * R + 0.587 * G + 0.114 * B
by = 0.5 * R + 0.5 * G - B
rg = R - G
row, col, dim = I.shape   # 等价于 MATLAB 的 [row col dim] = size(I)
CE_gray = np.zeros((row, col), dtype=np.float64)  # double(zeros(row, col))
CE_by = np.zeros((row, col), dtype=np.float64)
CE_rg = np.zeros((row, col), dtype=np.float64)

test(I , 'I')
test(R , 'R')
test(G , 'G')
test(B , 'B')
test(gray , 'gray')
test(by , 'by')
test(rg , 'rg')

输入变量形状: (768, 576, 3)
Mat变量形状: (768, 576, 3)
✓ 变量 'I' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'R' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'G' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'B' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'gray' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'by' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'rg' 数值相同


True

下面用来测试 border_in 函数

In [124]:
# 添加边界
def border_in(I, ps):
    """
    为输入图像添加边界
    
    参数:
    I: 输入图像 (2D numpy数组)
    ps: 补丁大小 (patch size)
    
    返回:
    nI: 添加边界后的图像
    """
    
    # 计算上下边界的复制行数
    if ps % 2 == 0:  # 偶数情况
        uc = ps // 2      # upperside copy
        dc = ps // 2 - 1  # downside copy
    else:  # mod(ps,2)==1 奇数情况
        uc = ps // 2      # floor(ps/2) 在Python中 // 就是向下取整
        dc = uc
        
    # test(uc , 'uc' , 'data/border_in_gray.mat')
    # test(dc , 'dc' , 'data/border_in_gray.mat')
    
    
    
    # 复制上边界和下边界
    ucb = I[:uc, :]           # I(1:uc,:) - 上边界
    dcb = I[-dc-1:, :]          # I(end-dc:end,:) - 下边界
    Igtemp1 = np.vstack([ucb, I, dcb])  # [ucb;I;dcb] - 垂直拼接
    # test(ucb , 'ucb' , 'data/border_in_gray.mat')
    # test(dcb , 'dcb' , 'data/border_in_gray.mat')
    # test(Igtemp1 , 'Igtemp1' , 'data/border_in_gray.mat')
    
    
    # 复制左边界和右边界
    lcb = Igtemp1[:, :uc]     # Igtemp1(:,1:uc) - 左边界
    rcb = Igtemp1[:, -dc-1:]    # Igtemp1(:,end-dc:end) - 右边界
    nI = np.hstack([lcb, Igtemp1, rcb])  # [lcb Igtemp1 rcb] - 水平拼接
    
    return nI


# gray_temp1 = border_in(gray, border_s)   


In [125]:
def border_out(I, ps):
    """
    移除图像边界
    
    参数:
    I: 输入图像 (2D numpy数组)
    ps: 补丁大小 (patch size)
    
    返回:
    nI: 移除边界后的图像
    """
    # 创建输入图像的副本，避免修改原始数据
    nI = I.copy()
    
    # 计算要移除的边界大小
    if ps % 2 == 0:  # mod(ps,2)==0
        uc = ps // 2      # upperside copy
        dc = ps // 2 - 1  # downside copy
    else:  # mod(ps,2)==1
        uc = ps // 2      # floor(ps/2)
        dc = uc
    
    # 移除边界 - 注意顺序很重要！
    # 先移除水平边界（左右）
    if uc > 0:
        nI = nI[:, uc:]           # 移除左边界 I(:,1:uc) = []
    if dc > 0:
        nI = nI[:, :-dc-1]          # 移除右边界 I(:,end-dc:end) = []
    
    # 再移除垂直边界（上下）
    if uc > 0:
        nI = nI[uc:, :]           # 移除上边界 I(1:uc,:) = []
    if dc > 0:
        nI = nI[:-dc-1, :]          # 移除下边界 I(end-dc:end,:) = []
    
    return nI

In [ ]:
# CE_Gray
gray_temp1 = border_in(gray, border_s)

Cx_gray = ndimage.convolve(gray_temp1, Gx, mode='constant')  # conv2(..., 'same')
Cy_gray = ndimage.convolve(gray_temp1, Gx.T, mode='constant')  # conv2(..., Gx', 'same')
C_gray_temp2 = np.sqrt(Cx_gray**2 + Cy_gray**2)
C_gray = border_out(C_gray_temp2, border_s)
R_gray = (C_gray * np.max(C_gray)) / (C_gray + (np.max(C_gray) * semisaturation))
R_gray_temp1 = R_gray - t1
mask = R_gray_temp1 > 0.0000001
CE_gray[mask] = R_gray_temp1[mask]


test(gray_temp1 , 'gray_temp1')
test(Cx_gray , 'Cx_gray')
test(Cy_gray , 'Cy_gray')
test(C_gray_temp2 , 'C_gray_temp2')
test(C_gray , 'C_gray')
test(R_gray , 'R_gray')
test(R_gray_temp1 , 'R_gray_temp1')
test(CE_gray , 'CE_gray')

输入变量形状: (788, 596)
Mat变量形状: (788, 596)
✓ 变量 'gray_temp1' 数值相同
输入变量形状: (788, 596)
Mat变量形状: (788, 596)
✓ 变量 'Cx_gray' 数值相同
输入变量形状: (788, 596)
Mat变量形状: (788, 596)
✓ 变量 'Cy_gray' 数值相同
输入变量形状: (788, 596)
Mat变量形状: (788, 596)
✓ 变量 'C_gray_temp2' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'C_gray' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'R_gray' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'R_gray_temp1' 数值相同
输入变量形状: (2, 398182)
Mat变量形状: (398182, 1)
形状不同：(2, 398182) vs (398182, 1)
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'CE_gray' 数值相同


True

In [129]:
# CE_by
by_temp1 = border_in(by, border_s)

Cx_by = ndimage.convolve(by_temp1, Gx, mode='constant')  # conv2(..., 'same')
Cy_by = ndimage.convolve(by_temp1, Gx.T, mode='constant')  # conv2(..., Gx', 'same')
C_by_temp2 = np.sqrt(Cx_by**2 + Cy_by**2)
C_by = border_out(C_by_temp2, border_s)
R_by = (C_by * np.max(C_by)) / (C_by + (np.max(C_by) * semisaturation))
R_by_temp1 = R_by - t2
mask = R_by_temp1 > 0.0000001
CE_by[mask] = R_by_temp1[mask]


test(by_temp1 , 'by_temp1')
test(Cx_by , 'Cx_by')
test(Cy_by , 'Cy_by')
test(C_by_temp2 , 'C_by_temp2')
test(C_by , 'C_by')
test(R_by , 'R_by')
test(R_by_temp1 , 'R_by_temp1')
test(CE_by , 'CE_by')

输入变量形状: (788, 596)
Mat变量形状: (788, 596)
✓ 变量 'by_temp1' 数值相同
输入变量形状: (788, 596)
Mat变量形状: (788, 596)
✓ 变量 'Cx_by' 数值相同
输入变量形状: (788, 596)
Mat变量形状: (788, 596)
✓ 变量 'Cy_by' 数值相同
输入变量形状: (788, 596)
Mat变量形状: (788, 596)
✓ 变量 'C_by_temp2' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'C_by' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'R_by' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'R_by_temp1' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'CE_by' 数值相同


True

In [131]:
# CE_rg
rg_temp1 = border_in(rg, border_s)

Cx_rg = ndimage.convolve(rg_temp1, Gx, mode='constant')  # conv2(..., 'same')
Cy_rg = ndimage.convolve(rg_temp1, Gx.T, mode='constant')  # conv2(..., Gx', 'same')
C_rg_temp2 = np.sqrt(Cx_rg**2 + Cy_rg**2)
C_rg = border_out(C_rg_temp2, border_s)
R_rg = (C_rg * np.max(C_rg)) / (C_rg + (np.max(C_rg) * semisaturation))
R_rg_temp1 = R_rg - t3
mask = R_rg_temp1 > 0.0000001
CE_rg[mask] = R_rg_temp1[mask]


test(rg_temp1 , 'rg_temp1')
test(Cx_rg , 'Cx_rg')
test(Cy_rg , 'Cy_rg')
test(C_rg_temp2 , 'C_rg_temp2')
test(C_rg , 'C_rg')
test(R_rg , 'R_rg')
test(R_rg_temp1 , 'R_rg_temp1')
test(CE_rg , 'CE_rg')

输入变量形状: (788, 596)
Mat变量形状: (788, 596)
✓ 变量 'rg_temp1' 数值相同
输入变量形状: (788, 596)
Mat变量形状: (788, 596)
✓ 变量 'Cx_rg' 数值相同
输入变量形状: (788, 596)
Mat变量形状: (788, 596)
✓ 变量 'Cy_rg' 数值相同
输入变量形状: (788, 596)
Mat变量形状: (788, 596)
✓ 变量 'C_rg_temp2' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'C_rg' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'R_rg' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'R_rg_temp1' 数值相同
输入变量形状: (768, 576)
Mat变量形状: (768, 576)
✓ 变量 'CE_rg' 数值相同


True